In [12]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation
from pgmpy.factors.discrete import DiscreteFactor

from utils import *
import pandas as pd
import numpy as np
import multiprocessing
import random

# Algoritmo genetico para obtener la mejor combinacion de medicamentos dada una patologia



** Setear los parametros del individuo (background, sintomas, efectos secundarios) **

In [40]:
class PathologyIndividual:
    "Clase destinada a las patologias, antecedentes del paciente, sintomas de la patologia y efectos de los medicamentos "

    def __init__(self, systemPathology, diccBackground, diccMedicines,arrStates,arrSymtopm, arrWeigth, bayesianNetwork):
        self.systemPathology = systemPathology
        self.diccBackground ={}
        self.diccMedicines  ={}
        
        self.arrStates       = arrStates[:]
        self.arrSymtopm      = arrSymtopm[:]
        self.arrWeigth       = arrWeigth[:]
        self.bayesianNetwork = bayesianNetwork
        
        for index in diccBackground:  
            self.diccBackground[index]  = diccBackground[index]
        
        for index in diccMedicines:           
            self.diccMedicines[index]  = diccMedicines[index]



**Clase abstracta de un individuo de algoritmo genético**

In [29]:
class Individual:
    "Clase abstracta para individuos de un algoritmo evolutivo."

    def __init__(self, chromosome):
        self.chromosome = chromosome

    def crossover(self, other):
        "Retorna un nuevo individuo cruzando self y other."
        raise NotImplementedError
        
    def mutate(self):
        "Cambia los valores de algunos genes."
        raise NotImplementedError

<b>Clase concreta de un individuo del problema de combinacion de medicamentos</b>

In [57]:
class Individual_medicines(Individual):
    "Clase que implementa el individuo en el problema de la busqueda de medicamentos que maximice la ausencia de los sintomas."

    def __init__(self, chromosome):
        self.chromosome = chromosome[:]
        self.fitness    = -1

    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre self y other "
        c = random.randrange(len(self.chromosome))
        ind1 = Individual_medicines(self.chromosome[:c] + other.chromosome[c:])
        ind2 = Individual_medicines(other.chromosome[:c] + self.chromosome[c:])
        return [ind1, ind2]   
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual_medicines(chromosome1)
        ind2 = Individual_medicines(chromosome2)
        return [ind1, ind2]     

    def mutate_position(self, arrStates):
        "Cambia aleatoriamente el estado del medicamento (prescribir, prescribir bajas condiciones, no prescribir)"
        for i,val in enumerate(self.chromosome):
            if (random.uniform(0, 1) < 0.25):
                new_value = changeState(val, arrStates[i])
                self.chromosome[i] = new_value
        return self.chromosome
    

<b>Funcion de fitness para evaluar un individuo del problema de combinacion de medicamentos</b>

In [64]:
def fitnessFunction(params):
    """Retorna el fitness de un cromosoma en el problema de combinacion de medicamentos """
    index, pathologyInd, individuo = params

    bp = BeliefPropagation(pathologyInd.bayesianNetwork)
    
    mergeSym         = pathologyInd.arrSymtopm  
    newDiccMedicines = setStatesChromosome(pathologyInd.diccMedicines, individuo)    
    evidence = createEvidence(pathologyInd.diccBackground,pathologyInd.diccMedicines) 
    print("Evidence: ",evidence)
    phi      = bp.query(variables=mergeSym, evidence = evidence)

    sumAbsen = 0
    sumTotal = 0
    
    for i,val in enumerate(mergeSym):
        arrProb = phi[val].values
        print("ArrProb: ",arrProb)
        large = len(arrProb)
        sumAbsen += arrProb[large-1]*pathologyInd.arrWeigth[i]
        sumTotal += 1*pathologyInd.arrWeigth[i]  
    fitness = sumAbsen / sumTotal
    individuo.fitness = fitness

    return [index, fitness]
    

<b>Funcion para evaluar toda una población de individuos con la funcion de fitnes especificada</b>

In [65]:
def evaluate_population(pathologyInd, population, fitness_fn,pool_size):
    """ Evalua una poblacion de individuos con la funcion de fitness pasada """
    popsize    = len(population)

    param_arr = []
    for i in range(popsize):
        if population[i].fitness == -1:  
            param_arr.append([i, pathologyInd, population[i]])

    pool = multiprocessing.Pool(pool_size)
    pool_results = pool.map(fitness_fn, param_arr)
    
    for fitness_arr in pool_results:
        population[fitness_arr[0]].fitness = fitness_arr[1]
    pool.close()
    pool.join()

<b>Funcion que selecciona con el metodo de torneo un par de individuos de population para cruzamiento </b>

In [53]:
def select_parents_tourney(population,ts):
    population_select = population.copy()
    # Escoje el primer padre
    iParent1 = chooseParent(population_select,ts)
    #Desconsiderar el padre ya escogido
    population_select.pop(iParent1)
    # Escoje el segundo padre
    iParent2 = chooseParent(population_select,ts)    
      
    return (population[iParent1], population[iParent2])

<b>Funcion que selecciona sobrevivientes para la sgte generacion, dada la poblacion actual y poblacion de hijos </b>

In [34]:
def select_survivors(population, offspring_population, numsurvivors):
    next_population = []
    population.extend(offspring_population) # une las dos poblaciones
    isurvivors = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:numsurvivors]
    for i in range(numsurvivors): next_population.append(population[isurvivors[i]])
    return next_population

<b>Algoritmo Genetico</b>   
Recibe una clase parametro para el individuo, funcion de fitness, numero de generaciones (ngen) y taza de mutación (pmut)

In [60]:
def genetic_algorithm(pathologyInd, population, fitness_fn, ngen=100, pmut=0.1,ts=2,pool_size=20):
    "Algoritmo Genetico "
    
    popsize    = len(population)

    evaluate_population(pathologyInd,population, fitness_fn,pool_size)
    ibest       = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
    bestfitness = [population[ibest[0]].fitness]
    print("Poblacion inicial, best_fitness = {}".format(population[ibest[0]].fitness))
    for g in range(ngen):
        
        ## Selecciona las parejas de padres para cruzamiento 
        mating_pool = []
        for i in range(int(popsize/2)): mating_pool.append(select_parents_tourney(population,ts)) 

        ## Crea la poblacion descendencia cruzando las parejas del mating pool con Recombinación de 1 punto
        offspring_population = []
        for i in range(len(mating_pool)): 
            offspring_population+= mating_pool[i][0].crossover_onepoint(mating_pool[i][1])
        
        ## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
        for i in range(len(offspring_population)):
            if random.uniform(0, 1) < pmut: 
                offspring_population[i].chromosome = offspring_population[i].mutate_position(pathologyInd.arrStates)

        ## Evalua la poblacion descendencia
        evaluate_population(pathologyInd, offspring_population, fitness_fn,pool_size) 
        
        ## Selecciona poblacion de individuos para la sgte. generación
        population = select_survivors(population, offspring_population, popsize)

        ## Almacena la historia del fitness del mejor individuo
        ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
        bestfitness.append(population[ibest[0]].fitness)
        print("generacion {}, best_fitness = {}".format(g, population[ibest[0]].fitness))
        print("Combinacion de medicamentos= {}".format(population[ibest[0]].chromosome))
    
    return population[ibest[0]], bestfitness  

 <b>Algoritmo de Busqueda Genetica para el problema de combinacion de medicamentos en adultos mayores</b>   

In [61]:
def genetic_search_medicines(ts,systemName, background, medicines, states, symtopms, weights, 
                             bayesianNetwork, fitness_fn, pool_size=20, num_medicines=11, popsize=20, ngen=50, pmut=0.25):
    import random
    population = []

    ## Crea la poblacion inicial con cromosomas aleatorios
    for i in range(popsize):
        chromosome = [random.randint(0,1) for j in range(1,(num_medicines+1))]
        random.shuffle(chromosome)
        population.append( Individual_medicines(chromosome) )
    
    pathologyInd = PathologyIndividual(systemName, background, medicines,states, symtopms, weights, bayesianNetwork)
    print("Sistema: ",pathologyInd.systemPathology)
    print("Background: ", pathologyInd.diccBackground)
    ## llama al algoritmo genetico para encontrar una solucion al problema de la combinacion de medicamentos
    return genetic_algorithm(pathologyInd, population, fitness_fn, ngen, pmut,ts,pool_size)
        

# Probando el Algoritmo genetico

In [ ]:
import matplotlib.pyplot as plt
from pgmpy.readwrite import BIFReader

reader = BIFReader('hipertension.bif')
bn0 = reader.get_model()
# busca solucion para el problema de medicamentos. Usa 10 individuos, 100 generaciones y taza de mutación de 0.25
best_ind, bestfitness = genetic_search_medicines(4, 'Cardiologia',{'EP_Insuficiencia_Cardiaca':0, 'EP_Asma_Bronquial':0}, {'MR_metropolol':0, 'MR_hidro':0,'MR_amlodipino':0,'MR_enalapril':0,'MR_captopril':0, 'MR_carvedilol':0,'MR_furosemida': 0,'MR_losartan':0, 'MR_prazosin':0,'MR_propanolol':0,'MR_verapamilo':0},[2,2,2,2,2,2,2,2,2,2,2],['SA_presion_arterial','SA_angina_de_pecho','SA_danho_rinhones'], [0.8,0,0],bn0,fitnessFunction, 10)
plt.plot(bestfitness)
plt.show()

[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1]
[1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1]
[0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1]
[0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0]
[0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1]
[1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0]
[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1]
[1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0]
[1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0]
[1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
[0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0]
Sistema:  Cardiologia
Background:  {'EP_Asma_Bronquial': 0, 'EP_Insuficiencia_Cardiaca': 0}
Evidence:  {'MR_captopril': 0, 'MR_metropolol': 0, 'MR_amlodipino': 1, 'MR_prazosin': 0, 'MR_propanolol': 0, 'MR_carvedilol': 1, 'EP_Asma_Bronquial': 0, 'MR_enalapril': 1, 'MR_hidro': 0, 'MR_verapamilo': 1, 'MR_losartan': 1, 'MR

In [ ]:
# API
#https://github.com/Akawa94/mkt-api/blob/dev/server.py